In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import median_absolute_error

In [ ]:
data = pd.read_csv("weatherHistory_shortened.csv")

In [ ]:
data["date"] = np.zeros(len(data))
for i in range(len(data)):
    data["date"][i] = data["Formatted Date"][i][0:10]


data["time"] = np.zeros(len(data))
for i in range(len(data)):
    data["time"][i] = data["Formatted Date"][i][11:13]
    data["time"]

In [ ]:
import datetime
data["nth_day"] = np.zeros(len(data))
for i in range(len(data)):
    datetime.datetime.strptime(data["date"][i] ,'%Y-%m-%d')
    data["nth_day"][i] = datetime.datetime.strptime(data["date"][i],'%Y-%m-%d').timetuple().tm_yday
    data["nth_day"] = data["nth_day"].to_numpy(int)

data["nth_day"]

In [ ]:
data['Date_new'] = pd.to_datetime(data['Formatted Date'], yearfirst=True)
data['Date_in_Days'] = data['Date_new'].map(lambda date:f'{30*(date.month-1) + date.day}') 
data['Date_in_Days'] = data['Date_in_Days'].to_numpy(dtype=int)
data

In [ ]:
data['date'] = pd.to_datetime(data['date'], yearfirst=True)
data['date']

In [ ]:
data = data.sort_values(by=['date'], ascending=True).reset_index()
data

In [ ]:
data = data.copy()
data["Average temperature"] = data["Temperature (C)"]
data.groupby("nth_day")["Average temperature"].mean().reset_index()

In [ ]:
plt.figure(figsize=(12,6))

plt.scatter(data['nth_day'].to_numpy(), data['Average temperature'].to_numpy(), marker='.')
plt.title("Weather history ")
plt.xlabel("Day")
plt.ylabel("Temperature (C)")
plt.show()

In [ ]:
X = data["nth_day"].copy().values
Y = data["Average temperature"].copy().values

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1) 
Y_train = Y_train.reshape(-1,1)
Y_test = Y_test.reshape(-1,1)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

In [ ]:
class Perceptron:
    def __init__(self, learning_rate, input_length, activation_func):
        self.learning_rate = learning_rate
        self.weights = np.random.rand(input_length)
        self.bias = np.random.rand(1)
        self.activation_function = activation_func     

    def activation(self, x, function):
        if function == "sigmoid":
            return 1 / (1 + np.exp(-x))
        elif function == "relu":
            return np.maximum(0, x)
        elif function == "tanh":
            return np.tanh(x)
        elif function == "linear":
            return x
        else:
            raise Exception("Unknown activation function")

    def fit(self, X_train, Y_train, X_test, Y_test, epochs):
        Loss_Train = []         
        Acc_Train = []          
        Loss_Test = []          
        Acc_Test = []           
        for epoch in tqdm(range(epochs)):
            for x, y in zip(X_train, Y_train):
                # forwarding 
                y_pred = x @ self.weights + self.bias
                y_pred = self.activation(y_pred , self.activation_function)     
                # y_pred = y_pred * self.activation(y_pred, "sigmoid")

                # back propagation
                error = y - y_pred

                # updating
                self.weights += self.learning_rate * error * x
                self.bias += self.learning_rate * error

            train_loss ,train_acc = model.evaluate(X_train , Y_train)           
            test_loss ,test_acc = model.evaluate(X_test , Y_test)               

            Loss_Train.append(train_loss)                                       
            Acc_Train.append(train_acc)                                         
            Loss_Test.append(test_loss)                                         
            Acc_Test.append(test_acc)                                           

        return Loss_Train, Acc_Train, Loss_Test, Acc_Test                      

    # def predict(self, X_test):
    #     Y_pred = []
    #     for x_test in X_test:
    #         y_pred = x_test @ self.weights + self.bias
    #         y_pred = self.activation(y_pred, "relu")
    #         Y_pred.append(y_pred)
    #     return np.array(Y_pred)
    
    def predict(self, X_test):
        y_pred = X_test @ self.weights + self.bias
        y_pred = self.activation(y_pred, "relu")

        return np.array(y_pred)

    def calculate_loss(self, X_test, Y_test, metric):
        Y_pred = self.predict(X_test)
        if metric == "mse":
            return np.mean(np.square(Y_test - Y_pred))
        elif metric == "mae":
            return np.mean(np.abs(Y_test - Y_pred))
        elif metric == "rmse":
            return np.sqrt(np.mean(np.square(Y_test - Y_pred)))
        else:
            raise Exception("Unknown metric")
        
    def calculate_accuracy(self, X_test, Y_test):
        Y_pred = self.predict(X_test)
        # Y_pred = Y_pred.reshape(-1)
        # Y_pred = np.where(Y_pred > 0.5, 1, 0)
        # accuracy = np.sum(Y_pred == Y_test) / len(Y_test)
        accuracy = median_absolute_error(Y_pred , Y_test)
        return accuracy

    def evaluate(self, X_test, Y_test):
        loss = self.calculate_loss(X_test, Y_test, "rmse")
        accuracy = self.calculate_accuracy(X_test, Y_test)

        return loss, accuracy
    
model = Perceptron(learning_rate=0.00001, input_length=X_train.shape[1], activation_func="relu")
Loss_Train, Acc_Train, Loss_Test, Acc_Test = model.fit(X_train, Y_train, X_test, Y_test, epochs=30)

In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
fig , (ax1 , ax2) = plt.subplots(1,2 , figsize=(12,5))


ax1.plot(Loss_Train)
ax1.plot(Loss_Test)
ax1.set_xlabel("epoch")
ax1.set_ylabel("loss")
ax1.legend(["train" , "test"])
ax1.title.set_text(f'Model loss (activation function: relu)')

ax2.plot(Acc_Train)
ax2.plot(Acc_Test)
ax2.set_xlabel("epoch")
ax2.set_ylabel("accuracy")
ax2.legend(["train" , "test"])
ax2.title.set_text('Model accuracy (activation function: relu)')

plt.show()

In [ ]:
model.predict(X_test[25])